## 建立迴歸變數

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
original_review=pd.read_csv('original_review.csv')
By_sentence_pos= pd.read_csv('BY_sentence_emo_score_detail.csv')

In [ ]:
emo_ = By_sentence_pos.groupby('old_index')['emotion_score'].sum()
emo_score = pd.DataFrame(columns=['total_score'])
emo_score['total_score']=emo_
for cat in classif.keys():
    emo_score[f'score_{cat}'] = By_sentence_pos[By_sentence_pos[f'word_in_{cat}'].isnull()==False].groupby('old_index')['emotion_score'].sum()
emo_score.reset_index(inplace=True)

In [ ]:
original_review['is_Hotel']=original_review['hotel_rank'].apply(lambda x:1 if '飯店' in x else 0)
original_review['is_BnB']=original_review['hotel_rank'].apply(lambda x:1 if '民宿' in x else 0)
original_review['is_Specialty_lodging']=original_review['hotel_rank'].apply(lambda x:1 if '特色住宿' in x else 0)
ch_list=['信義', '萬華', '大同', '大安', '中山', '中正', '松山', '北投', '南港', '士林','內湖', '文山']
en_list=['Xinyi','Wanhua','Datong','Daan','Zhongshan','Zhongzheng','Songshan','Beitou','Nangang','Shilin','Neihu','Wenshan']
for ch,en in zip(ch_list,en_list):
    original_review[f'is_{en}']=original_review['hotel_rank'].apply(lambda x:1 if ch in x else 0)
original_review['year']=original_review['date'].apply(lambda x:x[:4] if pd.isna(x)==False else 0)
for name in ['is_cp', 'is_place', 'is_service', 'is_comfort', 'is_clean', 'is_sleep']:
    original_review[name]=original_review[name]/10
    original_review=original_review.rename(columns={name:name[3:]+"_subrating"})
cn=['家庭旅行', '商務旅行', '單人旅行', '浪漫雙人遊', '好友出遊']
en=['family','business','single','double','friends']
for ch,en in zip(cn,en):
    original_review[f'Type_{en}']=original_review['trip'].apply(lambda x:0 if(( pd.isna(x))|(ch!=x))else 1 )
original_review['member_reviewsnum']=original_review['reviewsnum'].apply(lambda x: float(x.replace('篇投稿',"").replace(',','').strip()) if pd.isna(x)==False else None)
original_review['member_allhelpful']=original_review['allhelpful'].apply(lambda x:float(x.replace('人推薦',"").replace(',','').strip()) if pd.isna(x)==False else None)
original_review['hotel_total_rating']=original_review['hotel_total_rating']/10
original_review['rating']=original_review['rating']/10
original_review['hotel_star']=original_review['hotel_star'].apply(lambda x:float(0 if x== 'no_hotel_star' else x))
for ye in [2019., 2020., 2018., 2017., 2016., 2015., 2014., 2013., 2012.,
       2011., 2010., 2009., 2007., 2008.]:
        original_review[f'is_{int(ye)}']=original_review['year'].apply(lambda x:1 if (int(x)==ye) else 0)

original_review=original_review.rename(columns={'ReviewNum':'hotel_reviewsnum'}).drop(columns=['reviewsnum', 'member_name', 'place','allhelpful',
       'hotel_name','hotel_address','hotel_rank',
       'hotel_subrate_place', 'hotel_subrate_clean', 'hotel_subrate_service',
       'hotel_subrate_cp', 'hotel_facility','title',
       'auto_translate','date', 'trip', 'total_dict','year'])

In [ ]:
all_data=original_review.merge(emo_score,how='left')
all_data.drop(columns=['review'],inplace=True)

In [ ]:
all_data.to_csv('reg_8aspect_data.csv',index=False)

In [ ]:
#(各主題)字段有被分出主題
score_service = By_sentence_pos[By_sentence_pos['word_in_service'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_service=By_sentence_pos[By_sentence_pos['word_in_service'].isnull()==False].groupby('old_index').agg({'word_in_service': 'sum'}).reset_index()
df_score_service=score_service.merge(word_in_service,how='left')

score_place = By_sentence_pos[By_sentence_pos['word_in_place'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_place=By_sentence_pos[By_sentence_pos['word_in_place'].isnull()==False].groupby('old_index').agg({'word_in_place': 'sum'}).reset_index()
df_score_place=score_place.merge(word_in_place,how='left')

score_clean = By_sentence_pos[By_sentence_pos['word_in_clean'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_clean=By_sentence_pos[By_sentence_pos['word_in_clean'].isnull()==False].groupby('old_index').agg({'word_in_clean': 'sum'}).reset_index()
df_score_clean=score_clean.merge(word_in_clean,how='left')

score_cp = By_sentence_pos[By_sentence_pos['word_in_cp'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_cp=By_sentence_pos[By_sentence_pos['word_in_cp'].isnull()==False].groupby('old_index').agg({'word_in_cp': 'sum'}).reset_index()
df_score_cp=score_cp.merge(word_in_cp,how='left')

score_comfort = By_sentence_pos[By_sentence_pos['word_in_comfort'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_comfort=By_sentence_pos[By_sentence_pos['word_in_comfort'].isnull()==False].groupby('old_index').agg({'word_in_comfort': 'sum'}).reset_index()
df_score_comfort=score_comfort.merge(word_in_comfort,how='left')

score_sleep = By_sentence_pos[By_sentence_pos['word_in_sleep'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_sleep=By_sentence_pos[By_sentence_pos['word_in_sleep'].isnull()==False].groupby('old_index').agg({'word_in_sleep': 'sum'}).reset_index()
df_score_sleep=score_sleep.merge(word_in_sleep,how='left')

score_environment=By_sentence_pos[By_sentence_pos['word_in_environment'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_environment=By_sentence_pos[By_sentence_pos['word_in_environment'].isnull()==False].groupby('old_index').agg({'word_in_environment': 'sum'}).reset_index()
df_score_environment=score_environment.merge(word_in_environment,how='left')

score_food=By_sentence_pos[By_sentence_pos['word_in_food'].isnull()==False].groupby('old_index')['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count'].sum().reset_index()
word_in_food=By_sentence_pos[By_sentence_pos['word_in_food'].isnull()==False].groupby('old_index').agg({'word_in_food': 'sum'}).reset_index()
df_score_food=score_food.merge(word_in_food,how='left')


In [ ]:
df_score_service.rename(columns={'emotion_score':'service_score'},inplace=True)
df_score_place.rename(columns={'emotion_score':'place_score'},inplace=True)
df_score_clean.rename(columns={'emotion_score':'clean_score'},inplace=True)
df_score_cp.rename(columns={'emotion_score':'cp_score'},inplace=True)
df_score_comfort.rename(columns={'emotion_score':'comfort_score'},inplace=True)
df_score_sleep.rename(columns={'emotion_score':'sleep_score'},inplace=True)
df_score_environment.rename(columns={'emotion_score':'environment_score'},inplace=True)
df_score_food.rename(columns={'emotion_score':'food_score'},inplace=True)

In [ ]:
total_score_data=all_data.drop(columns=['score_service', 'score_place',
       'score_clean', 'score_cp', 'score_comfort', 'score_sleep',
       'score_environment', 'score_food'])

In [ ]:
total_score_data.merge(df_score_service,how='right').to_csv('result/BY_sentence_emo_score_service.csv',index=False)
total_score_data.merge(df_score_place,how='right').to_csv('result/BY_sentence_emo_score_place.csv',index=False)
total_score_data.merge(df_score_clean,how='right').to_csv('result/BY_sentence_emo_score_clean.csv',index=False)
total_score_data.merge(df_score_cp,how='right').to_csv('result/BY_sentence_emo_score_cp.csv',index=False)
total_score_data.merge(df_score_comfort,how='right').to_csv('result/BY_sentence_emo_score_comfort.csv',index=False)
total_score_data.merge(df_score_sleep,how='right').to_csv('result/BY_sentence_emo_score_sleep.csv',index=False)
total_score_data.merge(df_score_environment,how='right').to_csv('result/BY_sentence_emo_score_environment.csv',index=False)
total_score_data.merge(df_score_food,how='right').to_csv('result/BY_sentence_emo_score_food.csv',index=False)